In [1]:
import numpy as np
import cv2
import os
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping



<h4>1. Preprocessing of CT images and removal of black regions</h4>

In [2]:
def crop_black_borders(image, threshold=5):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mask = gray > threshold
    coords = np.argwhere(mask)
    if coords.size == 0:
        return image
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0) + 1
    return image[y0:y1, x0:x1]


<h4>2. Histogram extraction of intensity and gradient values over 10×10 pixel windows</h4>

In [3]:
def extract_features(image, gray_levels=8):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    h, w = gray.shape
    ph, pw = h // 10, w // 10
    feature_vector = []

    for i in range(10):
        for j in range(10):
            patch = gray[i*ph:(i+1)*ph, j*pw:(j+1)*pw]

            # Histogram of Intensity
            hist_intensity = cv2.calcHist([patch], [0], None, [gray_levels], [0, 256]).flatten()
            hist_intensity = hist_intensity / (hist_intensity.sum() + 1e-6)

            # Histogram of Gradient Magnitude
            gx = cv2.Sobel(patch, cv2.CV_32F, 1, 0, ksize=3)
            gy = cv2.Sobel(patch, cv2.CV_32F, 0, 1, ksize=3)
            mag = cv2.magnitude(gx, gy)
            hist_gradient = cv2.calcHist([mag.astype('uint8')], [0], None, [gray_levels], [0, 256]).flatten()
            hist_gradient = hist_gradient / (hist_gradient.sum() + 1e-6)

            feature_vector.extend(hist_intensity)
            feature_vector.extend(hist_gradient)

    return np.array(feature_vector)


<h4>3. Feature normalization</h4>

In [4]:
scaler = MinMaxScaler()

def normalize_features(X):
    return scaler.fit_transform(X)


<h4>4. Construction of an LSTM model</h4>

In [5]:
def build_lstm_model(input_shape, num_classes=4):
    model = Sequential()
    model.add(LSTM(200, input_shape=input_shape))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


<h4>5. Model training</h4>

In [6]:
def train_model(X, y, epochs=250):
    y_cat = to_categorical(y, num_classes=4)
    X_seq = X.reshape(X.shape[0], X.shape[1], 1)

    model = build_lstm_model((X_seq.shape[1], 1))
    
    early_stop = EarlyStopping(monitor='val_loss', patience=10)
    model.fit(X_seq, y_cat, epochs=epochs, batch_size=16, validation_split=0.2, callbacks=[early_stop])
    
    return model


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# مسیر دیتاست
DATASET_DIR = 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone'

# نگاشت کلاس‌ها به لیبل‌ها
label_map = {'Cyst': 0, 'Normal': 1, 'Stone': 2, 'Tumor': 3}

X_features = []
y_labels = []

# خواندن تصاویر از هر کلاس
for class_name, label in label_map.items():
    folder_path = os.path.join(DATASET_DIR, class_name)
    for filename in tqdm(os.listdir(folder_path), desc=f"Processing {class_name}"):
        if filename.lower().endswith('.jpeg'):  # فقط فایل‌های JPEG
            file_path = os.path.join(folder_path, filename)
            try:
                img = cv2.imread(file_path)
                if img is None:
                    continue
                cropped = crop_black_borders(img)
                resized = cv2.resize(cropped, (200, 200))  # اندازه استاندارد
                features = extract_features(resized)
                X_features.append(features)
                y_labels.append(label)
            except Exception as e:
                print(f"خطا در پردازش {file_path}: {e}")

X_features = np.array(X_features)
y_labels = np.array(y_labels)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\\Cyst'

In [ ]:
# نرمال‌سازی ویژگی‌ها
X_normalized = normalize_features(X_features)

# آموزش مدل LSTM
model = train_model(X_normalized, y_labels)

# ذخیره مدل نهایی
model.save("ct_kidney_classifier_lstm.h5")
